In [41]:
import mido

mid = mido.MidiFile("StorkeTower.mid", clip=True)


def roundPartial(value, resolution):
    return round(value / resolution) * resolution


patterns = []
prev_tempo = None
time = 0
for msg in mid:
    time += msg.time
    if msg.type == "program_change":
        print(msg)
    if msg.type == "set_tempo":
        print(msg)
        print("Tempo set to", msg.tempo)
        bpm = 6e7 / msg.tempo
        if prev_tempo != msg.tempo:
            if prev_tempo is not None:
                print("TIME", msg.time)
                length = time * patterns[-1]["bpm"] / 60 / 4
                print("LENGTH", length)
                patterns[-1]["time"] = time
                patterns[-1]["cumulative_time"] = (
                    time
                    if len(patterns) <= 1
                    else time + patterns[-2]["cumulative_time"]
                )
                patterns[-1]["length"] = roundPartial(length, 0.25)
                time = 0  # time resets every time tempo changes
            patterns.append({"bpm": bpm})
        print("bpm:", round(bpm, 1))
        prev_tempo = msg.tempo
else:
    length = time * patterns[-1]["bpm"] / 60 / 4
    print("LENGTH", length)
    patterns[-1]["time"] = time
    patterns[-1]["cumulative_time"] = (
        time if len(patterns) <= 1 else time + patterns[-2]["cumulative_time"]
    )
    patterns[-1]["length"] = roundPartial(length, 0.25)
    print("TIME", time)
    print("length", length)
print(patterns)

MetaMessage('set_tempo', tempo=1224489, time=0)
Tempo set to 1224489
bpm: 49.0
program_change channel=0 program=14 time=0
MetaMessage('set_tempo', tempo=1000000, time=0.00255101875)
Tempo set to 1000000
TIME 0.00255101875
LENGTH 5.750000000000001
bpm: 60.0
MetaMessage('set_tempo', tempo=1000000, time=0.0020833333333333333)
Tempo set to 1000000
bpm: 60.0
LENGTH 7.500000000000001
TIME 30.000000000000004
length 7.500000000000001
[{'bpm': 49.000031850020704, 'time': 28.163247000000005, 'cumulative_time': 28.163247000000005, 'length': 5.75}, {'bpm': 60.0, 'time': 30.000000000000004, 'cumulative_time': 58.16324700000001, 'length': 7.5}]


In [42]:
# Initialize lists to store MIDI data and output
midi_events = []
outputs = [[]]
i = 0

# Extract relevant MIDI events and convert them to dictionaries
for event in mid:
    midi_events.append(event.dict())

# Convert delta times to absolute times
current_time = 0
for event in midi_events:
    event["time"] += current_time
    current_time = event["time"]

    # Convert note_on events with 0 velocity to note_off events
    if event["type"] == "note_on" and event["velocity"] == 0:
        event["type"] = "note_off"

    # Prepare the event data for output
    event_data = []
    if event["type"] in ["note_on", "note_off"]:
        event_data = [event["type"], event["note"], event["time"], event["channel"]]
        if event["time"] < patterns[i]["cumulative_time"]:
            if i >= 1:
                event_data[2] -= patterns[i - 1]["cumulative_time"]
            outputs[i].append(event_data)
        else:
            i += 1
            event_data[2] -= patterns[i - 1]["cumulative_time"]
            outputs.append([event_data])
        print("event_data", event_data)

# Display the processed MIDI events
for i, output in enumerate(outputs):
    print("Pattern", i)
    for event in output:
        print(event)

event_data ['note_on', 60, 0, 0]
event_data ['note_off', 60, 1.22193798125, 0]
event_data ['note_on', 57, 1.224489, 0]
event_data ['note_off', 57, 2.44642698125, 0]
event_data ['note_on', 48, 2.4489780000000003, 0]
event_data ['note_off', 48, 4.8954049812500005, 0]
event_data ['note_on', 48, 4.897956000000001, 0]
event_data ['note_off', 48, 6.119893981250001, 0]
event_data ['note_on', 53, 6.122445000000001, 0]
event_data ['note_off', 53, 7.344382981250001, 0]
event_data ['note_on', 57, 7.346934000000001, 0]
event_data ['note_off', 57, 8.568871981250002, 0]
event_data ['note_on', 50, 8.571423000000001, 0]
event_data ['note_off', 50, 9.79336098125, 0]
event_data ['note_on', 57, 9.795912, 0]
event_data ['note_off', 57, 12.24233898125, 0]
event_data ['note_on', 59, 12.24489, 0]
event_data ['note_off', 59, 13.466827981249999, 0]
event_data ['note_on', 57, 13.469378999999998, 0]
event_data ['note_off', 57, 15.91580598125, 0]
event_data ['note_on', 60, 15.918356999999999, 0]
event_data ['note

In [43]:
def generate_tracks(output):
    # create a 31-element list of empty lists called "tracks"
    tracks = [[] for _ in range(31)]

    # put all notes in the right track
    for event in output:
        tracks[event[3]].append(event)

    # put all notes in the right order
    for track in tracks:
        track.sort(key=lambda event: event[2])

    # print all tracks
    for track in tracks:
        print(track)
    print()

    return tracks


tracks_list = []
for output in outputs:
    tracks_list.append(generate_tracks(output))

[['note_on', 60, 0, 0], ['note_off', 60, 1.22193798125, 0], ['note_on', 57, 1.224489, 0], ['note_off', 57, 2.44642698125, 0], ['note_on', 48, 2.4489780000000003, 0], ['note_off', 48, 4.8954049812500005, 0], ['note_on', 48, 4.897956000000001, 0], ['note_off', 48, 6.119893981250001, 0], ['note_on', 53, 6.122445000000001, 0], ['note_off', 53, 7.344382981250001, 0], ['note_on', 57, 7.346934000000001, 0], ['note_off', 57, 8.568871981250002, 0], ['note_on', 50, 8.571423000000001, 0], ['note_off', 50, 9.79336098125, 0], ['note_on', 57, 9.795912, 0], ['note_off', 57, 12.24233898125, 0], ['note_on', 59, 12.24489, 0], ['note_off', 59, 13.466827981249999, 0], ['note_on', 57, 13.469378999999998, 0], ['note_off', 57, 15.91580598125, 0], ['note_on', 60, 15.918356999999999, 0], ['note_off', 60, 17.140294981249998, 0], ['note_on', 52, 17.142846, 0], ['note_off', 52, 18.36478398125, 0], ['note_on', 55, 18.367335, 0], ['note_off', 55, 19.58927298125, 0], ['note_on', 53, 19.591824000000003, 0], ['note_of

In [44]:
import uuid


def format_tracks(tracks, bpm):
    tracks_with_duration = [[] for _ in range(31)]

    # calculate the duration of each note
    for i in range(len(tracks)):
        for j in range(len(tracks[i])):
            if tracks[i][j][0] == "note_on":
                for k in range(j, len(tracks[i])):
                    if (
                        tracks[i][k][0] == "note_off"
                        and tracks[i][k][1] == tracks[i][j][1]
                    ):
                        tracks_with_duration[i].append(
                            {
                                "id": str(uuid.uuid4()),
                                "pitch": float(tracks[i][j][1]),
                                "time": round(tracks[i][j][2] * bpm / 60 / 4, 2),
                                "duration": round(
                                    (tracks[i][k][2] - tracks[i][j][2]) * bpm / 60 / 4, 2
                                ),
                            }
                        )
                        break

    # print all tracks with duration
    for i in tracks_with_duration:
        print(i)
    print()

    return tracks_with_duration


tracks_with_duration_list = []
for tracks, pattern in zip(tracks_list, patterns):
    tracks_with_duration_list.append(format_tracks(tracks, pattern["bpm"]))

[{'id': 'ef763649-3e6a-4d97-bd49-7d4795717877', 'pitch': 60.0, 'time': 0.0, 'duration': 1.0}, {'id': '6a1408cf-6429-45a4-9942-6959287a5491', 'pitch': 57.0, 'time': 1.0, 'duration': 1.0}, {'id': '08698be1-ed80-49e3-84c7-2d6a8d72af40', 'pitch': 48.0, 'time': 2.0, 'duration': 2.0}, {'id': '8de133ab-dd94-4779-8bce-d7a504a40d61', 'pitch': 48.0, 'time': 4.0, 'duration': 1.0}, {'id': '5001d49f-0699-44e8-9ba6-5ee3daabc761', 'pitch': 53.0, 'time': 5.0, 'duration': 1.0}, {'id': 'a402cd23-49d8-456a-a340-efc052e93f7e', 'pitch': 57.0, 'time': 6.0, 'duration': 1.0}, {'id': 'c24510f2-f40c-4592-b858-f0cd09da391b', 'pitch': 50.0, 'time': 7.0, 'duration': 1.0}, {'id': '1dee5713-6346-41e0-a954-375e1b658953', 'pitch': 57.0, 'time': 8.0, 'duration': 2.0}, {'id': '33659f34-f1ed-408f-aed0-a5e43d388971', 'pitch': 59.0, 'time': 10.0, 'duration': 1.0}, {'id': 'c6ca00e9-ad0e-43a5-87e2-7f2f535acbf1', 'pitch': 57.0, 'time': 11.0, 'duration': 2.0}, {'id': 'a7004511-90ca-41aa-b670-90555d24d3f1', 'pitch': 60.0, 'time

In [45]:
total_time = 0
for msg in mid:
    total_time += msg.time

print("Total time:", total_time)

size = round(
    total_time * bpm / 60 / 4
)  # because 60 seconds in a minute (we use bpm <- minute) and 4 beats in a bar

print("Size:", size)

Total time: 58.163247000000005
Size: 15


In [46]:
# program_change channel=0 program=36 time=0
# program_change channel=1 program=0 time=0
# program_change channel=2 program=32 time=0
# program_change channel=3 program=0 time=0
# program_change channel=4 program=81 time=0
# program_change channel=5 program=80 time=0
# program_change channel=6 program=81 time=0
# program_change channel=7 program=42 time=0
# program_change channel=8 program=64 time=0
# program_change channel=9 program=0 time=0
# program_change channel=10 program=64 time=0
# program_change channel=11 program=64 time=0
# program_change channel=12 program=64 time=0
# program_change channel=13 program=64 time=0
# program_change channel=14 program=64 time=0
# program_change channel=15 program=64 time=0
channel_patches = [
    35,
    31,
    30,
    31,
    9,
    80,
    5,
    16,
    11,
    33,
    11,
    11,
    11,
    11,
    11,
    11,
]

In [47]:
res = {
    "value0": {
        "patterns": [
            {
                "bpm": round(patterns[i]["bpm"], 1),
                "length": patterns[i]["length"],
                "name": "",
                "id": str(uuid.uuid4()),
                "tracks": tracks_with_duration_list[i],
            }
            for i in range(len(patterns))
        ],
        "channels": [
            {
                "patch": patch,
                "atk": 0.0010000000474974514,
                "rel": 0.12999999523162843,
                "volume": 1.0,
                "pan": 0.0,
                "lp": 20000.0,
                "tune": 0.0,
                "soft": True,
            }
            for patch in channel_patches
        ],
    }
}

In [48]:
# output to song.json
import json

with open("song.json", "w") as f:
    json.dump(res, f, indent=4)
    